### Ragas is a framework that helps you evaluate your Retrieval Augmented Generation (RAG) pipelines


In [1]:
import os, json
from langchain.chains import AnalyzeDocumentChain
from langchain_openai import ChatOpenAI

from utils import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY 
from agentic_chunker import AgenticChunker
from llm_utils import *
from llm_utils import _text
#os.environ["LANGCHAIN_TRACING_V2"] = "true"

#openai.api_key = os.environ['OPENAI_API_KEY']

[nltk_data] Downloading package punkt_tab to /home/oleg/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/oleg/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  

In [3]:
#loader_url =SeleniumURLLoader(["https://arxiv.org/pdf/2312.10997",
pdf_list = ["./data/data_rag/s41524-023-01062-z.pdf",
    "./data/data_rag/s41699-023-00369-1.pdf"
         #   "https://github.com/HSE-LAMBDA/ai4material_design/tree/main/docs/CONSTRUCTOR-MOCK.md"
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/CONSTRUCTOR.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/DATA.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/ENVIRONMENT.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-CONSTRUCTOR.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-MOCK.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/PILOT.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/SPARSE-PAPER.md"
          #  "https://www.nature.com/articles/s41377-024-01407-3",
          #  "https://www.nature.com/articles/s41565-023-01407-1",
          #  "https://www.nature.com/articles/s41699-023-00369-1",
           ]
                               
chunks, documents = load_pdf_documets(pdf_list, tokens=1000)
print(len(chunks), len(documents))

Created a chunk of size 1135, which is longer than the specified 1000


128 35


### Prompts

In [4]:
from langchain_core.prompts import PromptTemplate

agent_chunking = PromptTemplate.from_template("""
You are an expert in analyzing scientific articles.
You are given "Text":
###
{text}
--------------
Your task is to split this "Text" into chunks united by common meaning and ideas,
consisting of sentences from this text. 
Each chunk should have one common idea or hypothetical question.
The total number of words in each chunk should be greater than {Nmin}, but less than {Nmax}.
Do not rewrite the text, keep the original spelling and sentences.
All these chunks should contain all information from this "Text".

Present the results as a list of strings, each element of the list is one chunk of splited text, formatted in JSON.

Example:
['Two-dimensional materials offer a promising platform for the next generation of (opto-) 
electronic devices and other high technology applications. One of the most exciting characteristics of 
2D crystals is the ability to tune their properties via controllable introduction of defects. 
However, the search space for such structures is enormous, and ab-initio computations prohibitively expensive. 
We propose a machine learning approach for rapid estimation of the properties of 2D material given 
the lattice structure and defect configuration. The method suggests a way to represent configuration of 2D materials
with defects that allows a neural network to train quickly and accurately. 
We compare our methodology with the state-of-the-art approaches and demonstrate at least 3.7 times 
energy prediction error drop.',  'The search space for such structures is enormous, 
and ab-initio computations prohibitively expensive.']
""")



In [5]:
from langchain_core.output_parsers import JsonOutputParser

In [6]:
#agent2 = agent_chunking2 | llm | JsonOutputParser()
agent = agent_chunking | llm | JsonOutputParser()

In [7]:
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel

class Documets(BaseModel):
    page_content: str
    metadata: dict
    #def __init__(self, page_content: str, metadata: dict):
    #    self.page_content = page_content
    #    self.metadata = metadata
    #def __str__(self):
    #    return self.page_content
    #def __repr__(self):
    #    return f"Documets(page_content={self.page_content}, metadata={self.metadata})"
#new_docs = 

### Make vectores DB

In [16]:
from tqdm import tqdm
from pymilvus import MilvusClient

milvus_client = MilvusClient("./milvus_demo.db")

### Split document by LLM

In [10]:

article_propositions = []
for d in documents:
    answer_agent = agent.invoke({'text': d.page_content, "Nmin": 600, "Nmax": 1200})
    #print(answer_agent)
    #print(len(answer_agent), 10*'*** ')
    #try:
    #    chs = [" ".join(answer_agent)]
   # 
    #except TypeError:
    #    chs = [" ".join(a) for a in answer_agent]
    article_propositions += [Documets(page_content=s, metadata=d.metadata) for s in answer_agent]
    
    print(d.metadata)

{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 0}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 0}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 1}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 1}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 2}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 2}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 3}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 3}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 4}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 5}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 5}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 6}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 7}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 7}
{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 8}
{'source': './data/data_rag/s41524-023-0

In [11]:
print(len(article_propositions))
print(article_propositions[0].page_content, "\n\n")
print(article_propositions[-1].page_content)

67
Two-dimensional materials offer a promising platform for the next generation of (opto-) electronic devices and other high technology applications. One of the most exciting characteristics of 2D crystals is the ability to tune their properties via controllable introduction of defects. However, the search space for such structures is enormous, and ab-initio computations prohibitively expensive. We propose a machine learning approach for rapid estimation of the properties of 2D material given the lattice structure and defect configuration. The method suggests a way to represent configuration of 2D materials with defects that allows a neural network to train quickly and accurately. We compare our methodology with the state-of-the-art approaches and demonstrate at least 3.7 times energy prediction error drop. Also, our approach is an order of magnitude more resource-efficient than its contenders both for the training and inference part. Atomic-scale tailoring of materials is one of the m

### Create chunks by union 

In [12]:
from agentic_chunker import AgenticChunker
ac = AgenticChunker(openai_api_key=OPENAI_API_KEY)

/home/oleg/miniconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [13]:
ac.add_propositions([d.page_content for d in article_propositions])


Adding: 'Two-dimensional materials offer a promising platform for the next generation of (opto-) electronic devices and other high technology applications. One of the most exciting characteristics of 2D crystals is the ability to tune their properties via controllable introduction of defects. However, the search space for such structures is enormous, and ab-initio computations prohibitively expensive. We propose a machine learning approach for rapid estimation of the properties of 2D material given the lattice structure and defect configuration. The method suggests a way to represent configuration of 2D materials with defects that allows a neural network to train quickly and accurately. We compare our methodology with the state-of-the-art approaches and demonstrate at least 3.7 times energy prediction error drop. Also, our approach is an order of magnitude more resource-efficient than its contenders both for the training and inference part. Atomic-scale tailoring of materials is one o

/home/oleg/miniconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


55964
Chunk Found (55964), adding to: Two-Dimensional Materials Engineering

Adding: 'In this work, we propose a method for predicting the energetic and electronic structures of defects in 2D materials with machine learning. Firstly, a machine learning-friendly 2D Material Defect database (2DMD) was established employing high throughput DFT calculations. The database is composed of both structured datasets and dispersive datasets of defects in represented 2D materials such as MoS2, WSe2, h-BN, GaSe, InSe, and black phosphorous (BP). We use the datasets to evaluate the performance of the previously reported approaches along with ours, which was specially designed to provide accurate description of materials with defects. Our computational experiments show that our approach provides a significant increase in prediction accuracy compared to the state-of-the-art general methods. The high accuracy allows to reproduce the nonlinear non-monotonic property-distance correlation of defects which

55964
Chunk Found (55964), adding to: Machine Learning for Defect Engineering in 2D Materials

Adding: 'Sparse representation of crystals with defectsFor machine learning algorithms, an atomic structure is a so-called point cloud: a set of points in 3D space. Each point is associated with a vector of properties, which at the least contains the atomic number, but may also include more physics-based features, such as radius, the number of valence electrons, etc. The structures with defects present a challenge to machine learning algorithms. The neighborhoods of the majority of the atoms are not affected by the point defects. In principle, this shouldn’t be an obstacle for a perfect algorithm. In practice, however, this comparatively small difference in the full structures is hard to learn. As we demonstrate in section “Aggregate performance” with our computational experiments, state-of-the-art algorithms underperform on crystal structures with defects. We propose a way to represent struc

55964
Chunk Found (55964), adding to: Machine Learning for Analyzing Defects in 2D Materials

Adding: 'We split the dataset into 3 parts: train (60%), validation (20%), and test (20%). The split is random and stratiﬁed with respect to each base material. For each model, we use random search for hyperparameter optimization; we generate 50 hyperparameter conﬁgurations, train the model with each conﬁguration on the train part, and select the best-performing conﬁguration by evaluating quality on the validation part. The search spaces and optimal conﬁgurations are present in Supplementary Discussion 1. To obtain the ﬁnal result, we train each model with the optimal parameters on the combination of train and validation parts and evaluate the quality on the unseen test part. We do this 12 times to estimate the effects of the random initialization. We use unrelaxed structures as inputs and predict the energy and HOMO –LUMO gap after relaxation. To account for the material class imbalance, we u

...

None
No chunks found
Created new chunk (e58e3): Material Properties

Adding: 'Table 1 continued Type Defect site Count Variation range Structure Mo S S Energy (eV) HOMO (meV) LUMO (meV) V4 vac Se 15 0.197 129 25 V5 vac Se Se 743 0.430 245 73 V6 vac vac Se 1415 2.282 203 236 S1 W 1 S2 W vac 15 0.042 15 24 S3 W vac vac 743 0.193 108 394 S4 W Se 15 0.028 <10 <10 S5 W Se Se 743 0.079 <10 <10 P. Huang et al. 4 npj 2D Materials and Applications (2023) 6 Published in partnership with FCT NOVA with the support of E-MRS'
e58e3
Chunk Found (e58e3), adding to: Material Properties

Adding: 'widest range of ~4.1 eV, whereas defects that included one Mo vacancy and only one S vacancy varies within a moderate range of~2.2 eV. The variation in the formation energy of defects containing two S vacancies is only 0.1 eV. The variation of the formation energy of defects composed mainly by substitution atoms is only a few tens of meV. Such a spread of energies for different types of defects seems logical. T

No chunks
No chunks found
Created new chunk (2f75b): Quantum Properties of Materials

Adding: 'In metals, the introduction of impurities generally results in Friedel oscillations 29. Due to the strong screening of the surrounding electrons, the long-range tail of a charged impurity potential is suppressed, resulting in a power law decay of the defect disturbance30,31. In insulators, however, when the wave function is strongly localized on the defect site —the lattice structure plays a signi ﬁcant role in the defect properties1,32,33. In crystals with a honeycomb structure, the interaction between defects is strongly in ﬂuenced by the presence of the two sublattices. Thus, it has been predicted and con ﬁrmed both by theory and experiment that the coupling between hydrogen atoms on the surfaces of graphene depends on the sublattices occupied 34,35. To understand the defect interaction in TMDCs, we employ a simpli ﬁed two-orbital picture. We propose that there are two localized defect orb

None
No chunks found
Created new chunk (ec198): Material Properties

Adding: 'We develop a database of machine learning-friendly datasets on the physical properties of solid-state materials with and withoutdefects. As a starting point, a structured dataset of 11,866 defectconﬁgurations in TMDCs and a dispersive dataset of 3000conﬁgurations in six represented 2D materials were created. It isbased on high throughput DFT calculations and unveils thecomplex structure-property correlations through proper dataordering and physical insights. The initially structured datasetspans all possible single, double, and triple defects con ﬁgurationswith components of Mo/W and S/Se vacancies, W/Mo, and Se/Ssubstitutions in an 8 × 8 × 1 supercell. According to the band gapvs formation energy map, the property distribution of the defectconﬁgurations was visualized. This may provide a general guide-line for defect engineering in TMDCs to enable emergingtechnologies. The property of a selected double and t

e58e3
Chunk Found (e58e3), adding to: Defect Energies and Electronic Properties in MoS2

Adding: 'Jain, A. et al. Commentary: The Materials Project: a materials genome approach to accelerating materials innovation. APL Mater. 1, 011002 (2013). Kirklin, S. et al. The open quantum materials database (OQMD): assessing the accuracy of DFT formation energies. npj Comput. Mater. 1,1–15 (2015). Saal, J. E., Kirklin, S., Aykol, M., Meredig, B. & Wolverton, C. Materials design and discovery with high-throughput density functional theory: the open quantum materials database (OQMD). JOM 65, 1501 –1509 (2013). Curtarolo, S. et al. AFLOW: An automatic framework for high-throughput materials discovery. Comput. Mater. Sci. 58, 218 –226 (2012). Chen, C., Ye, W., Zuo, Y., Zheng, C. & Ong, S. P. Graph networks as a universal machine learning framework for molecules and crystals. Chem. Mater. 31, 3564 –3572 (2019). Xie, T. & Grossman, J. C. Crystal graph convolutional neural networks for an accurate and 

None
No chunks found
Created new chunk (e37a5): Publication Details & Licensing


In [14]:
len(ac.get_chunks())

30

In [15]:
ks = list(ac.get_chunks().keys())
ac.get_chunks()[ks[0]]

{'chunk_id': '55964',
 'propositions': ['Two-dimensional materials offer a promising platform for the next generation of (opto-) electronic devices and other high technology applications. One of the most exciting characteristics of 2D crystals is the ability to tune their properties via controllable introduction of defects. However, the search space for such structures is enormous, and ab-initio computations prohibitively expensive. We propose a machine learning approach for rapid estimation of the properties of 2D material given the lattice structure and defect configuration. The method suggests a way to represent configuration of 2D materials with defects that allows a neural network to train quickly and accurately. We compare our methodology with the state-of-the-art approaches and demonstrate at least 3.7 times energy prediction error drop. Also, our approach is an order of magnitude more resource-efficient than its contenders both for the training and inference part. Atomic-scale 

In [17]:
with open("2ai4mat_articles.json", "w+") as f:
    json.dump(ac.get_chunks(), f, indent=4)

### Load test QA  dataset

In [18]:
import pandas as pd

df_ground_true = pd.read_csv("./data/data_rag/QA_ai4mat_2articles.csv")

questions = df_ground_true['question'].values.tolist()
answers = df_ground_true['answer'].values.tolist()


### Compare 4 stratigies 
#### Recursive chunking,  Agentic chunking, LLM chunking, Markdown  chunking

In [19]:
from ragas.metrics import (
    context_utilization,
    context_entity_recall,
    context_relevancy,
    answer_relevancy,
    answer_correctness, 
    faithfulness,
    context_precision,
    context_recall,
)

from datasets import Dataset 

from ragas.evaluation import evaluate
import nest_asyncio

In [20]:
metrics = [
   answer_relevancy,
    answer_correctness, 
   faithfulness,
    context_precision,
    context_recall,
 #   context_entity_recall,
    context_relevancy,
 #   context_utilization,
]

In [21]:
emb, ch_list, dataset, result = dict(), dict(), dict(), dict()

### Embeedding of chunks

#### Recursive chunking

In [22]:
name = 'recursive'
emb[name], _, ch_list[name] = get_embs(chunks, embedding=OpenAIEmbeddings(),  question=False) 

len(emb[name]), len(emb[name][0]), len(ch_list[name])==len(emb[name])

(128, 1536, True)

In [23]:
collection_name = f"ai4mat_OpenAiEmb_{name}"

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=len(emb[name][0]),    # emb dimension
    metric_type="IP",            # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    )
print("dimension",len(emb[name][0]))

dimension 1536


In [24]:
data = []

for i, line in enumerate(tqdm(ch_list[name], desc="Creating embeddings")):
    data.append({"id": i, "vector": emb[name][i], "text": _text(ch_list[name][i]), })



milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|█████████████████████████| 128/128 [00:00<00:00, 277452.67it/s]


{'insert_count': 128,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127],
 'cost': 0}

In [25]:
dataset[name] =  get_datasamples(questions, 
                            answers, 
                            milvus_client, 
                            collection_name, 
                            embedding=lambda q: OpenAIEmbeddings().embed_query(q), 
                            llm = llm,
                            top_k=10)


0it [00:00, ?it/s]

In [26]:
nest_asyncio.apply()

#query_engine_bge = build_query_engine(hf_bge, documents)
result[name] = evaluate(dataset[name], metrics, llm=llm,  raise_exceptions=False, embeddings=OpenAIEmbeddings())

Evaluating:   0%|          | 0/210 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from 

#### Agentic chunking

In [27]:
name = 'agentic'
emb[name], _, ch_list[name] = get_embs(ac.get_chunks(get_type='list_of_strings'), embedding=OpenAIEmbeddings(),  question=False) 

len(emb[name]), len(emb[name][0]), len(ch_list[name])==len(emb[name])

(30, 1536, True)

In [28]:
collection_name = f"ai4mat_OpenAiEmb_{name}"

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=len(emb[name][0]),    # emb dimension
    metric_type="IP",            # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    )
print("dimension",len(emb[name][0]))

dimension 1536


In [29]:
data = []

for i, line in enumerate(tqdm(ch_list[name], desc="Creating embeddings")):
    data.append({"id": i, "vector": emb[name][i], "text": _text(ch_list[name][i]), })

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|████████████████████████████| 30/30 [00:00<00:00, 60436.66it/s]


{'insert_count': 30,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 'cost': 0}

In [30]:
dataset[name] =  get_datasamples(questions, 
                            answers, 
                            milvus_client, 
                            collection_name, 
                            embedding=lambda q: OpenAIEmbeddings().embed_query(q), 
                            llm = llm,
                            top_k=10)


0it [00:00, ?it/s]

In [31]:
nest_asyncio.apply()

#query_engine_bge = build_query_engine(hf_bge, documents)
result[name] = evaluate(dataset[name], metrics, llm=llm,  raise_exceptions=False, embeddings=OpenAIEmbeddings())

Evaluating:   0%|          | 0/210 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
Failed to parse output. Returning None.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.


#### LLM chunking

In [32]:
name = 'llm'
emb[name], _, ch_list[name] = get_embs(article_propositions, embedding=OpenAIEmbeddings(),  question=False) 

In [33]:
len(emb[name]), len(emb[name][0])

(67, 1536)

In [34]:
collection_name = f"ai4mat_OpenAiEmb_{name}"

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=len(emb[name][0]),    # emb dimension
    metric_type="IP",            # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    )
print("dimension",len(emb[name][0]))

dimension 1536


In [35]:
data = []

for i, line in enumerate(tqdm(ch_list[name], desc="Creating embeddings")):
    data.append({"id": i, "vector": emb[name][i], "text": _text(ch_list[name][i]), })

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|███████████████████████████| 67/67 [00:00<00:00, 124069.92it/s]


{'insert_count': 67,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66],
 'cost': 0}

In [36]:
dataset[name] =  get_datasamples(questions, 
                            answers, 
                            milvus_client, 
                            collection_name, 
                            embedding=lambda q: OpenAIEmbeddings().embed_query(q), 
                            llm = llm,
                            top_k=10)


0it [00:00, ?it/s]

In [37]:
nest_asyncio.apply()

#query_engine_bge = build_query_engine(hf_bge, documents)
result[name] = evaluate(dataset[name], metrics, llm=llm,  raise_exceptions=False, embeddings=OpenAIEmbeddings())

Evaluating:   0%|          | 0/210 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
Failed to parse output. Returning None.
No statements were generated from the answer.


### Use Markdown files

In [38]:
import pathlib, pathlib

def pdf2md(pdf_file):
    """
    Convert a PDF file to Markdown
    """
    
    md_file = pdf_file.replace('.pdf', '.md')
    if not pathlib.Path(md_file).exists():
        md_text = pymupdf4llm.to_markdown(pdf_file)
        # now work with the markdown text, e.g. store as a UTF8-encoded file
        pathlib.Path(md_file).write_bytes(md_text.encode())
    return md_file

In [39]:
article_md = []

for pdf_file in pdf_list:
    md_file = pdf2md(pdf_file)
    d = TextLoader(md_file).load()
    text =  d[0].page_content.split('\n\n')
    article_md += [Documets(page_content=a, metadata=d[0].metadata) for a in text]

In [40]:
len(article_md)

187

#### Markdown chunking

In [41]:
name = 'markdown'
emb[name], _, ch_list[name] = get_embs(article_md, embedding=OpenAIEmbeddings(),  question=False) 

In [42]:
len(emb[name]), len(emb[name][0])

(187, 1536)

In [43]:
collection_name = f"ai4mat_OpenAiEmb_{name}"

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=len(emb[name][0]),    # emb dimension
    metric_type="IP",            # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    )
print("dimension",len(emb[name][0]))

dimension 1536


In [44]:
data = []

for i, line in enumerate(tqdm(ch_list[name], desc="Creating embeddings")):
    data.append({"id": i, "vector": emb[name][i], "text": _text(ch_list[name][i]), })

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|█████████████████████████| 187/187 [00:00<00:00, 117327.58it/s]


{'insert_count': 187,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186],
 'cost': 0}

In [47]:
dataset[name] =  get_datasamples(questions, 
                            answers, 
                            milvus_client, 
                            collection_name, 
                            embedding=lambda q: OpenAIEmbeddings().embed_query(q), 
                            llm = llm,
                            top_k=10)


0it [00:00, ?it/s]

[search] retry:4, cost: 0.27s, reason: <_InactiveRpcError: StatusCode.UNAVAILABLE, failed to connect to all addresses; last error: UNKNOWN: unix:/tmp/tmpg89ynw21_milvus_demo.db.sock: Connection refused>
[search] retry:5, cost: 0.81s, reason: <_InactiveRpcError: StatusCode.UNAVAILABLE, failed to connect to all addresses; last error: UNKNOWN: unix:/tmp/tmpg89ynw21_milvus_demo.db.sock: Connection refused>
[search] retry:6, cost: 2.43s, reason: <_InactiveRpcError: StatusCode.UNAVAILABLE, failed to connect to all addresses; last error: UNKNOWN: unix:/tmp/tmpg89ynw21_milvus_demo.db.sock: Connection refused>
[search] retry:7, cost: 3.00s, reason: <_InactiveRpcError: StatusCode.UNAVAILABLE, failed to connect to all addresses; last error: UNKNOWN: unix:/tmp/tmpg89ynw21_milvus_demo.db.sock: Connection refused>
[search] retry:8, cost: 3.00s, reason: <_InactiveRpcError: StatusCode.UNAVAILABLE, failed to connect to all addresses; last error: UNKNOWN: unix:/tmp/tmpg89ynw21_milvus_demo.db.sock: Conne

KeyboardInterrupt: 

In [ ]:
nest_asyncio.apply()

#query_engine_bge = build_query_engine(hf_bge, documents)
result[name] = evaluate(dataset[name], metrics, llm=llm,  raise_exceptions=False, embeddings=OpenAIEmbeddings())

### Results

In [48]:
def print_dict(d, indent=0):
    """
    Recursively prints a dictionary, including nested dictionaries, 
    with indentation to represent structure.
    """
    for key, value in d.items():
        print('  ' * indent + str(key) + ": ", end="")
        if isinstance(value, dict):
            print()  # Print a newline for nested dictionaries
            print_dict(value, indent + 1)
        else:
            print(str(value))

In [49]:
print_dict(result)

recursive: 
  answer_relevancy: 0.22500236588587363
  answer_correctness: 0.27295712224694957
  faithfulness: 1.0
  context_precision: 0.47254850086277905
  context_recall: 0.5166666666666666
  context_relevancy: 0.011739134460718158
agentic: 
  answer_relevancy: 0.38147697449807216
  answer_correctness: 0.35999946873791044
  faithfulness: 0.9588235294117647
  context_precision: 0.5294753401186089
  context_recall: 0.6492063492063492
  context_relevancy: 0.01041445845618771
llm: 
  answer_relevancy: 0.35604953456005745
  answer_correctness: 0.3496116392963482
  faithfulness: 0.9230769230769231
  context_precision: 0.6389756325641388
  context_recall: 0.5666666666666667
  context_relevancy: 0.02315647646067191


In [50]:
df_m = pd.DataFrame().from_dict(result, orient='index')

df_m

,answer_relevancy,answer_correctness,faithfulness,context_precision,context_recall,context_relevancy
recursive,0.225002,0.272957,1.000000,0.472549,0.516667,0.011739
agentic,0.381477,0.359999,0.958824,0.529475,0.649206,0.010414
llm,0.356050,0.349612,0.923077,0.638976,0.566667,0.023156


In [ ]:
df_m.to_csv('df_compare_4spliters.csv')

In [ ]:
pd.DataFrame(dataset['llm'])

In [ ]:
pd.DataFrame(dataset['agentic'])